In [1]:
import numpy as np
import pandas as pd
import collections as col
from itertools import combinations
from itertools import permutations
from itertools import product
import plotly.express as px
from scipy.stats import ttest_ind
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()


In [2]:
df = pd.read_csv('D:\\Downloads\\stud_math.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

In [3]:
for col in df.columns:
    if df[col].dtype==type(object()):
        df[col]=df[col].apply(lambda x : x if isinstance(x,str) else '')
        df[col]=df[col].apply(lambda x: None if x.strip() == '' else x)
    if df[col].dtype!=type(object()):
        print(col)

      






age
Medu
Fedu
traveltime
studytime
failures
studytime, granular
famrel
freetime
goout
health
absences
score


In [4]:
'''строит по каждому столбцу гистограмму содержащихся типов'''
data_set=df
number=len(data_set.columns)
fig=make_subplots(rows=1,cols=number)
fig.update_layout(width=1600,height=800,showlegend=True)
objects=[go.Histogram(x=pd.Series(data_set.values[:,i]).apply(lambda x : str(type(x))),marker_line_color='white',marker_line_width=0.1,
                     name=data_set.columns[i]) 
         for i in range(number)]
fig.add_traces(objects,
              rows=[1]*number,
              cols=[i for i in range(1,number+1,1)])

In [5]:
'''строим распеределение признаков для столбцов с числовыми значениями'''
objects=[go.Box(y=pd.Series(df.values[:,i]),name=df.columns[i]) for i,col in enumerate(df.columns) 
         if df[col].dtype!=type(object())]
fig=make_subplots(rows=1,cols=len(objects))
fig.update_layout(height=800,width=1600,showlegend=True)
fig.add_traces(objects,rows=[1]*len(objects),cols=[i for i in range(1,len(objects)+1,1)])











In [6]:
'''построим распеределние нечисловых значений'''
objects = [go.Histogram(x=pd.Series(df.values[:, i]),   
                        name=df.columns[i]) for i, col in
           enumerate(df.columns) if df[col].dtype==type(object)]
fig= make_subplots(rows=1, cols=len(objects),subplot_titles=[col for col in df.columns if df[col].
                                                             dtype==type(object())])
fig.update_layout(height=800, width=1600, showlegend=True)
fig.add_traces(objects, rows=[1]*len(objects),
               cols=[i for i in range(1, len(objects)+1, 1)])

In [8]:
l=[col for col in df.columns if df[col].dtype!=type(object)]
l=list(combinations(l,2))
fig=make_subplots(cols=1,rows=78)
fig.update_layout(width=800,height=78000,showlegend=True)
fig.add_traces([go.Scatter(x=df[comb[0]],y=df[comb[1]],mode='markers') for i,comb in enumerate(l)],
              rows=[i for i in range(1,79)],
              cols=[1]*78
             )



     
        

In [10]:
def rows_cols(rows,cols,a):
    c=[]
    r=[]
    for i in range(1,rows+1):
        for k in range(1,cols+1):
            r.append(i)
            c.append(k)
    if a=='r':
        return r
    if a=='c':
        return c
rows_cols(3,2,'r')

        


[1, 1, 2, 2, 3, 3]

In [7]:
df_digital=pd.DataFrame([df[col] for col in df.columns if df[col].dtype!=type(object())]).T
fig = px.scatter_matrix(df_digital)
fig.update_layout(width=1600,height=1600)
fig.show()


In [19]:
df_digital.corr()

,age,Medu,Fedu,traveltime,studytime,failures,"studytime, granular",famrel,freetime,goout,health,absences,score
age,1.000000,-0.161704,-0.138018,0.081508,-0.015556,0.228047,0.015556,0.068899,0.029023,0.135107,-0.065946,0.093592,-0.155726
Medu,-0.161704,1.000000,0.275598,-0.173515,0.078589,-0.245463,-0.078589,-0.010050,0.025335,0.064164,-0.045051,-0.011944,0.213349
Fedu,-0.138018,0.275598,1.000000,0.030646,0.057199,-0.150821,-0.057199,0.040577,-0.066556,-0.023282,0.027251,0.004695,0.078573
traveltime,0.081508,-0.173515,0.030646,1.000000,-0.102206,0.064699,0.102206,-0.016857,-0.021937,0.037636,-0.021531,-0.048215,-0.070559
studytime,-0.015556,0.078589,0.057199,-0.102206,1.000000,-0.178992,-1.000000,0.045894,-0.121969,-0.051280,-0.080016,-0.026544,0.115741
failures,0.228047,-0.245463,-0.150821,0.064699,-0.178992,1.000000,0.178992,-0.022655,0.098382,0.120914,0.081284,0.057391,-0.344338
"studytime, granular",0.015556,-0.078589,-0.057199,0.102206,-1.000000,0.178992,1.000000,-0.045894,0.121969,0.051280,0.080016,0.026544,-0.115741
famrel,0.068899,-0.010050,0.040577,-0.016857,0.045894,-0.022655,-0.045894,1.000000,0.175743,0.081154,0.077474,-0.082609,0.057112
freetime,0.029023,0.025335,-0.066556,-0.021937,-0.121969,0.098382,0.121969,0.175743,1.000000,0.296437,0.061575,0.009292,0.019026
goout,0.135107,0.064164,-0.023282,0.037636,-0.051280,0.120914,0.051280,0.081154,0.296437,1.000000,-0.015608,-0.048738,-0.129685
